# Perform VADER Sentiment on Bitcoin and Litecoin to determine news stories sentiment

In [1]:
# Initial imports
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/mrnagleJR97/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Reading the News API key enviroment variable
api_key = os.getenv("news_api")



In [4]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)



In [5]:
# Fetch all the news about Facebook Libra
crypto_headlines = newsapi.get_everything(
    q="litecoin",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Litecoin: {crypto_headlines['totalResults']}")

# Show sample article
crypto_headlines["articles"][0]



Total articles about Litecoin: 499


{'source': {'id': 'the-verge', 'name': 'The Verge'},
 'author': 'Kim Lyons',
 'title': 'Litecoin price jumps 20 percent after hoax links the cryptocurrency and Walmart',
 'description': 'A fake press release that stated Walmart was partnering with Litecoin and would begin accepting the cryptocurrency was picked up by several news outlets Monday.',
 'url': 'https://www.theverge.com/2021/9/13/22671485/walmart-crypto-hoax-litecoin-cryptocurrency',
 'urlToImage': 'https://cdn.vox-cdn.com/thumbor/J6Pz0pi4qSyPcpQklTPayuKmfLE=/0x719:5000x3337/fit-in/1200x630/cdn.vox-cdn.com/uploads/chorus_asset/file/22846606/1234956057.jpg',
 'publishedAt': '2021-09-13T14:46:33Z',
 'content': 'An apparently fake press release said the retailer would begin accepting the cryptocurrency\r\nLitecoin spiked Monday after a false press release claimed Walmart would accept the cryptocurrency\r\nPhoto … [+1049 chars]'}

In [6]:
# Create the Facebook Libra sentiment scores DataFrame
crypto_sentiments = []

for article in crypto_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        crypto_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
crypto_df = pd.DataFrame(crypto_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
crypto_df = crypto_df[cols]

crypto_df.head()



,date,text,compound,positive,negative,neutral
0,2021-09-13,An apparently fake press release said the reta...,0.2732,0.152,0.090,0.758
1,2021-08-23,PayPal will now allow users outside the U.S. t...,0.4215,0.098,0.000,0.902
2,2021-08-23,"PayPal is bringing the ability to buy, hold an...",0.6908,0.161,0.000,0.839
3,2021-09-13,"Editor's Note: With so much market volatility,...",0.3365,0.064,0.000,0.936
4,2021-09-13,"""Don't worry about people stealing your ideas....",0.1546,0.177,0.123,0.700


In [7]:
# Get descriptive stats from the DataFrame
crypto_df.describe()



,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.00000
mean,0.125280,0.069040,0.042020,0.88893
std,0.422681,0.069595,0.067113,0.08486
min,-0.893400,0.000000,0.000000,0.64700
25%,-0.063350,0.000000,0.000000,0.83875
50%,0.141300,0.069500,0.000000,0.90650
75%,0.510600,0.094250,0.070250,0.94200
max,0.872000,0.275000,0.353000,1.00000


In [8]:
crypto_df.to_csv('./input_data/ltc_sentiment.csv')